# ESM2-650M LoRA 微调与嗜热性改造工作流

本 Notebook 实现了基于 LoRA (Low-Rank Adaptation) 对 ESM2-650M 模型进行参数高效微调的完整流程。

**核心目标：**
1. 利用嗜热蛋白序列微调模型，使其习得“耐热风格”。
2. 对比微调前后模型对特定突变的打分（LLR），筛选潜在的增稳突变。

**硬件说明：**
- 代码自动适配 CUDA (Linux/Windows), MPS (Mac), CPU。
- 建议显存 > 12GB。如果显存不足，请降低 `BATCH_SIZE` 或使用 Gradient Accumulation。

In [1]:
%cd /plaxy

/plaxy


### 1. 全局参数设置 (User Configuration)
**请在此处修改所有关键参数。** 这是整个脚本的控制台。

In [2]:
import os

# ===========================
# 1. [最优先] 设置 HuggingFace 镜像
# ===========================
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"

# ===========================
# 2. 项目配置参数 (Excel 数据源模式)
# ===========================

# 模型选择
MODEL_NAME = "facebook/esm2_t33_650M_UR50D"

# --- [NEW] 数据文件设置 ---
# Excel 文件路径
METADATA_FILE = "uniprot_ph.xlsx" 

# Excel 列名映射 (请根据你的 Excel 表头修改)
COL_ENTRY = "Entry"          # Uniprot ID 列
COL_SEQUENCE = "Sequence"    # 序列内容列
COL_MIN_PH = "min_ph"        # 最小 pH 列
COL_MAX_PH = "max_ph"        # 最大 pH 列
SHEET_NAME = 0               # Sheet索引 (0) 或名称

# --- [NEW] 筛选参数 ---
# 逻辑：保留 (min_ph <= FILTER_MIN_PH) 或 (max_ph >= FILTER_MAX_PH) 的数据
# 如果设为 None，则忽略该条件
FILTER_MIN_PH = 5.0
FILTER_MAX_PH = None

# 输出目录
OUTPUT_DIR = "./esm2_lora_ph5.0_less"

# 全局随机种子
SEED = 42

# ===========================
# 3. 训练超参数
# ===========================
LORA_RANK = 8                # LoRA 的秩，数据量少建议 8 或 16
LORA_ALPHA = 16              # 缩放系数，通常为 Rank 的 1-2 倍
LORA_DROPOUT = 0.05
LEARNING_RATE = 1e-4
NUM_EPOCHS = 3
BATCH_SIZE = 1
GRAD_ACCUMULATION = 16

# ===========================
# 4. 序列处理
# ===========================
MIN_SEQ_LENGTH = 30
MAX_SEQ_LENGTH = 1024
MLM_PROBABILITY = 0.15

### 2. 环境初始化与依赖检查
导入必要的库，设置随机种子，并自动检测可用的计算设备 (CUDA/MPS/CPU)。

In [3]:
import torch
import random
import numpy as np
from transformers import AutoTokenizer, EsmForMaskedLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset

# 检查 PEFT 是否安装
try:
    import peft
except ImportError:
    !pip install peft -q

# 1. 设置随机种子函数
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(SEED)

# 2. 设备检测
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")

device = get_device()
print(f"🔥 正在使用计算设备: {device}")
if device.type == 'cuda':
    print(f"   GPU型号: {torch.cuda.get_device_name(0)}")

🔥 正在使用计算设备: mps


### 3. 数据准备 (Data Preparation)
读取 FASTA 文件并将其转换为 HuggingFace Dataset 格式。
*注：如果指定路径下没有 FASTA 文件，这里会生成一个假的 FASTA 文件用于演示。*

In [4]:
import pandas as pd
import numpy as np
from datasets import Dataset

# 检查 pandas 和 openpyxl 是否安装
try:
    import pandas
    import openpyxl
except ImportError:
    print("正在安装必要的库 pandas openpyxl ...")
    !pip install pandas openpyxl -q

# --- 辅助函数：清洗 pH 值 ---
def clean_ph_value(val):
    """
    将输入转换为 float，处理非数值情况。
    """
    if pd.isna(val):
        return pd.NA
    try:
        # 尝试直接转换
        return float(val)
    except (ValueError, TypeError):
        # 如果是字符串，尝试处理复杂情况（如 '5.5' 或带空格）
        # 这里做一个简单的清洗，如果无法转换则返回 NA
        try:
            import re
            # 提取第一个浮点数
            match = re.search(r"[-+]?\d*\.\d+|\d+", str(val))
            if match:
                return float(match.group())
            return pd.NA
        except:
            return pd.NA

# --- 1. 读取 Excel 数据 ---
if not os.path.exists(METADATA_FILE):
    # 如果文件不存在，生成一个模拟的 Excel 用于测试代码运行
    print(f"⚠️ 未找到 {METADATA_FILE}，正在生成模拟 Excel 数据以供演示...")
    dummy_data = {
        COL_ENTRY: [f"P{i}" for i in range(100)],
        COL_SEQUENCE: [
            "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG",
            "MSIGTGYTPAIYNGSLKQGKPVISLSIGDTVTIDQKADIYAKLDNLVAQGKTFVIIADEAYA"
        ] * 50,
        COL_MIN_PH: [0.0, 4.0, 6.0, 2.0, None] * 20,
        COL_MAX_PH: [4.0, 6.0, 9.0, 10.0, 8.0] * 20
    }
    df = pd.DataFrame(dummy_data)
    # 临时保存以便流程继续，实际使用请确保路径正确
    METADATA_FILE = "dummy_data.xlsx" 
    df.to_excel(METADATA_FILE, index=False)
    print(f"✅ 模拟数据已保存至 {METADATA_FILE}")
else:
    print(f"📂 正在读取数据: {METADATA_FILE}")

# 读取 Excel
# 获取文件扩展名
file_ext = os.path.splitext(METADATA_FILE)[-1].lower()

try:
    if file_ext in ['.xlsx', '.xls']:
        # 读取 Excel
        print("   检测到 Excel 格式")
        df = pd.read_excel(METADATA_FILE, sheet_name=SHEET_NAME)
        
    elif file_ext == '.tsv':
        # 读取 TSV (Tab 分隔)
        print("   检测到 TSV 格式")
        df = pd.read_csv(METADATA_FILE, sep='\t')
        
    elif file_ext == '.csv':
        # 读取 CSV (逗号分隔)
        print("   检测到 CSV 格式")
        df = pd.read_csv(METADATA_FILE)
        
    else:
        raise ValueError(f"❌ 不支持的文件格式: {file_ext}。仅支持 .xlsx, .xls, .tsv, .csv")

    print(f"📊 原始数据加载成功，共 {len(df)} 行。")

except Exception as e:
    raise RuntimeError(f"❌ 读取文件失败: {e}")

# --- 2. 应用筛选逻辑 (User Defined Logic) ---

# 预处理：生成临时的数值型 pH 列
if COL_MIN_PH and COL_MIN_PH in df.columns:
    df['temp_min_ph'] = df[COL_MIN_PH].apply(clean_ph_value)
else:
    df['temp_min_ph'] = pd.NA

if COL_MAX_PH and COL_MAX_PH in df.columns:
    df['temp_max_ph'] = df[COL_MAX_PH].apply(clean_ph_value)
else:
    df['temp_max_ph'] = pd.NA

# 初始化筛选掩码 (Mask)
if FILTER_MIN_PH is None and FILTER_MAX_PH is None:
    mask = pd.Series([True] * len(df), index=df.index)
    print("💡 提示: 未设置任何 pH 筛选条件，保留所有数据。")
else:
    mask = pd.Series([False] * len(df), index=df.index)
    filter_applied = False

    # 条件 A: min_ph <= FILTER_MIN_PH
    if FILTER_MIN_PH is not None and 'temp_min_ph' in df.columns:
        condition_min = (df['temp_min_ph'].notna()) & (df['temp_min_ph'] <= FILTER_MIN_PH)
        mask = mask | condition_min  # 逻辑 OR
        print(f"✅ 筛选条件 A 应用: {COL_MIN_PH} <= {FILTER_MIN_PH}")
        filter_applied = True

    # 条件 B: max_ph >= FILTER_MAX_PH
    if FILTER_MAX_PH is not None and 'temp_max_ph' in df.columns:
        condition_max = (df['temp_max_ph'].notna()) & (df['temp_max_ph'] >= FILTER_MAX_PH)
        mask = mask | condition_max  # 逻辑 OR
        print(f"✅ 筛选条件 B 应用: {COL_MAX_PH} >= {FILTER_MAX_PH}")
        filter_applied = True

    # 应用筛选
    original_count = len(df)
    df = df[mask]
    print(f"📉 pH 筛选结果: {original_count} -> {len(df)} (保留满足任一条件的条目)")

# --- 3. 数据清洗 (去除无效序列) ---
# 确保序列列存在且不为空
if COL_SEQUENCE not in df.columns:
    raise ValueError(f"错误: Excel 中找不到序列列 '{COL_SEQUENCE}'")

# 去除 NaN 序列
df = df.dropna(subset=[COL_SEQUENCE])
# 强制转换为字符串
df[COL_SEQUENCE] = df[COL_SEQUENCE].astype(str)
# 简单的去重（可选，视需求而定，这里建议保留去重以防过拟合）
# df = df.drop_duplicates(subset=[COL_SEQUENCE])

# 提取序列列表
raw_sequences = df[COL_SEQUENCE].tolist()
print(f"📋 最终用于训练的有效序列数: {len(raw_sequences)}")

# 转换为 Dataset 对象
dataset = Dataset.from_dict({"text": raw_sequences})

# ===========================
# [保持原有] 长度过滤 (Data Cleaning)
# ===========================
MIN_LEN = MIN_SEQ_LENGTH
MAX_LEN = MAX_SEQ_LENGTH

print(f"🧹 正在过滤长度: 保留 {MIN_LEN} 到 {MAX_LEN} 之间的序列...")
original_count = len(dataset)
dataset = dataset.filter(lambda x: MIN_LEN <= len(x["text"]) <= MAX_LEN)
filtered_count = len(dataset)
print(f"📉 长度过滤结果: {original_count} -> {filtered_count} (剔除了 {original_count - filtered_count} 条)")

# 划分训练集和验证集 (90/10)
if len(dataset) > 0:
    dataset_split = dataset.train_test_split(test_size=0.1, seed=SEED)
    train_dataset = dataset_split["train"]
    eval_dataset = dataset_split["test"]
    print(f"🚆 训练集大小: {len(train_dataset)}")
    print(f"🧪 验证集大小: {len(eval_dataset)}")
else:
    raise ValueError("❌ 错误：经过筛选后没有剩余数据！请检查 Excel 数据或放宽筛选条件。")

📂 正在读取数据: uniprot_ph.xlsx
   检测到 Excel 格式
📊 原始数据加载成功，共 5462 行。
✅ 筛选条件 A 应用: min_ph <= 5.0
📉 pH 筛选结果: 5462 -> 716 (保留满足任一条件的条目)
📋 最终用于训练的有效序列数: 716
🧹 正在过滤长度: 保留 30 到 1024 之间的序列...


Filter:   0%|          | 0/716 [00:00<?, ? examples/s]

📉 长度过滤结果: 716 -> 668 (剔除了 48 条)
🚆 训练集大小: 601
🧪 验证集大小: 67


In [5]:
# 序列长度分布统计
import numpy as np
import matplotlib.pyplot as plt

# 提取所有序列长度
seq_lens = [len(x["text"]) for x in dataset] # 假设 dataset 是刚才生成的 Dataset 对象

# 计算分位数
p90 = np.percentile(seq_lens, 90)
p95 = np.percentile(seq_lens, 95)
p99 = np.percentile(seq_lens, 99)

print(f"平均长度: {np.mean(seq_lens):.1f}")
print(f"90% 的蛋白长度小于: {p90:.0f}")
print(f"95% 的蛋白长度小于: {p95:.0f}")
print(f"99% 的蛋白长度小于: {p99:.0f}")

平均长度: 417.4
90% 的蛋白长度小于: 676
95% 的蛋白长度小于: 785
99% 的蛋白长度小于: 924


### 4. 序列分词 (Tokenization)
使用 ESM2 的 tokenizer 对序列进行处理。我们需要设置 `truncation` 来处理过长的序列。

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(
        examples["text"], 
        padding=False,
        # padding="max_length",  # --- IGNORE --- 
        truncation=True, 
        max_length=MAX_SEQ_LENGTH
    )

print("⏳ 正在进行分词处理...")
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)
print("✅ 分词完成。")

⏳ 正在进行分词处理...


Map:   0%|          | 0/601 [00:00<?, ? examples/s]

Map:   0%|          | 0/67 [00:00<?, ? examples/s]

✅ 分词完成。


### 5. 模型加载与 LoRA 配置 (Model & LoRA Setup)
加载 ESM2-650M 模型，并应用 LoRA 适配器。
**注意**：这里我们只微调 `query`, `key`, `value` 层，这是最节省资源且有效的方式。

In [7]:
# 加载 Masked Language Model 头部的 ESM2
model = EsmForMaskedLM.from_pretrained(MODEL_NAME)

# 配置 LoRA
peft_config = LoraConfig(
    task_type=TaskType.FEATURE_EXTRACTION, # 或使用 generic task configuration
    inference_mode=False,
    r=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    # 针对 ESM2 架构，微调注意力机制的 Q/K/V
    target_modules=["query", "key", "value", "dense"], 
    bias="none"
)

# 将 LoRA 挂载到模型上
model = get_peft_model(model, peft_config)

# 打印可训练参数量，确认微调规模
model.print_trainable_parameters()

model.to(device)
print("🚀 模型已加载至显存并挂载 LoRA。")

trainable params: 6,103,040 || all params: 657,146,294 || trainable%: 0.9287
🚀 模型已加载至显存并挂载 LoRA。


### 6. 开始训练 (Training)
使用 HuggingFace `Trainer` 进行训练。使用 `DataCollatorForLanguageModeling` 自动进行随机 Masking (MLM 任务)。

In [8]:
# ===========================
# 补全缺失的 Data Collator 定义
# ===========================
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=True, 
    mlm_probability=MLM_PROBABILITY,
    pad_to_multiple_of=8  # <--- ⚡️ 新增：自动填充到当前 batch 最长序列，且对齐为 8 的倍数（加速计算）
)

# ===========================
# 训练参数配置 (已修复参数名和 MPS 问题)
# ===========================
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUMULATION,
    # === ⚡️ 新增神器 ⚡️ ===
    group_by_length=True,              # 自动把长度相似的蛋白凑一组，极大减少 Padding 浪费
    
    learning_rate=LEARNING_RATE,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",        # 使用新版参数名 eval_strategy
    load_best_model_at_end=True,  # 训练结束加载最好的模型
    fp16=torch.cuda.is_available(), # 如果是 CUDA 开启混合精度
    # use_mps_device=(device.type == 'mps'), # 已移除，新版自动处理
    seed=SEED,
    report_to="none" # 不上传 wandb
)

# ===========================
# 初始化 Trainer
# ===========================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator, # 这里就不会报错了
)

print("🏁 开始训练...")
trainer.train()

# 保存最终的 LoRA 权重
save_path = os.path.join(OUTPUT_DIR, "final_model")
model.save_pretrained(save_path)
print(f"✅ 训练完成，模型已保存至 {save_path}")

🏁 开始训练...


/Users/zhanghaonan/miniforge3/envs/oper/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,1.442300,1.392439
2,1.479800,1.386320
3,1.432400,1.309498


/Users/zhanghaonan/miniforge3/envs/oper/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/zhanghaonan/miniforge3/envs/oper/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


✅ 训练完成，模型已保存至 ./esm2_lora_ph5.0_less/final_model


### 7. 应用演示：对比打分 (In silico Screening)
这是本流程的核心：**计算突变的相对似然度 (LLR Score)**。

我们将对比同一突变在 **原始模型 (Base)** 和 **微调模型 (Fine-tuned)** 下的概率差值。
**技术点**：使用 `model.disable_adapter()` 上下文管理器，我们可以在不卸载/重载模型的情况下，瞬间切换回原始 ESM2 进行推理，极大节省显存。

In [14]:
def calculate_mutation_score(seq, position_1based, mutant_aa, model, tokenizer, device):
    """
    计算特定位点突变的 LLR 分数。
    参数:
        position_1based: 生物学位置，从 1 开始计数 (例如第 1 个氨基酸输入 1)
    """
    # 1. 边界检查与基础信息获取
    seq_len = len(seq)
    if position_1based < 1 or position_1based > seq_len:
        raise ValueError(f"位置 {position_1based} 超出序列范围 (1-{seq_len})")
    
    # 获取野生型氨基酸 (Python string 索引需要 -1)
    wt_aa = seq[position_1based - 1] 
    
    # 2. 准备模型输入
    model.eval()
    inputs = tokenizer(seq, return_tensors="pt", truncation=True, max_length=MAX_SEQ_LENGTH).to(device)
    
    # 获取突变氨基酸的 Token ID
    mut_token_id = tokenizer.convert_tokens_to_ids(mutant_aa)
    
    # 3. 确定 Tensor 中的索引位置
    # ESM2 的 input_ids 格式为: [CLS, AA1, AA2, ..., AAn, EOS]
    # 索引 0 是 CLS
    # 索引 1 是 AA1 (对应 position_1based=1)
    # 所以 Tensor 索引 = position_1based
    tensor_index = position_1based 
    
    with torch.no_grad():
        # --- 微调模型 (Fine-tuned) ---
        outputs_tuned = model(**inputs)
        log_probs_tuned = torch.nn.functional.log_softmax(outputs_tuned.logits, dim=-1)
        score_tuned = log_probs_tuned[0, tensor_index, mut_token_id].item()
        
        # --- 原始模型 (Base) ---
        with model.disable_adapter():
            outputs_base = model(**inputs)
            log_probs_base = torch.nn.functional.log_softmax(outputs_base.logits, dim=-1)
            score_base = log_probs_base[0, tensor_index, mut_token_id].item()
            
    llr_score = score_tuned - score_base
    return wt_aa, score_tuned, score_base, llr_score

# --- 演示 ---
# 假设这是你的待改造蛋白
target_protein = "VDNKFNKEQQNAFYEILHLPNLNEEQRNAFIQSLKDDPSQSANLLAEAKKLNDAQAPK"

# 【用户修改区】 设置符合生物学直觉的参数
test_pos = 36      # 直接写第 26 位
test_mut = "A"     # 突变成 R

# 计算
wt, st, sb, llr = calculate_mutation_score(target_protein, test_pos, test_mut, model, tokenizer, device)

# 生成标准突变名称 (如 A26R)
mutation_name = f"{wt}{test_pos}{test_mut}"

print(f"🔍 突变分析: {mutation_name}")
print(f"   位点: {test_pos} | 野生型: {wt} -> 突变型: {test_mut}")
print(f"   ---------------------------")
print(f"   原始模型得分 (LogProb): {sb:.4f}")
print(f"   微调模型得分 (LogProb): {st:.4f}")
print(f"   >>> LLR 提升 (Score): {llr:.4f}")

if llr > 0:
    print(f"🎉 推荐: {mutation_name} 可能是增稳突变！")
else:
    print(f"❄️ 不推荐: 模型对 {mutation_name} 反应消极。")

🔍 突变分析: D36A
   位点: 36 | 野生型: D -> 突变型: A
   ---------------------------
   原始模型得分 (LogProb): -3.5073
   微调模型得分 (LogProb): -3.4917
   >>> LLR 提升 (Score): 0.0156
🎉 推荐: D36A 可能是增稳突变！


### 8. 批量单点饱和突变扫描 (In silico DMS)
对指定区域进行全氨基酸扫描，寻找 LLR 最高的突变。

In [15]:
# 标准氨基酸列表
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

# 【用户修改区】 设置扫描范围 (1-based, 包含两端)
scan_start = 1   # 从第 5 个氨基酸开始
scan_end = 10    # 到第 10 个氨基酸结束

results = []
print(f"🔄 正在扫描区域 {scan_start} - {scan_end} (1-based)...")

# 循环范围：range(start, end + 1) 以包含结束位点
for pos in range(scan_start, scan_end + 1):
    # 简单的边界保护
    if pos > len(target_protein): break
    
    # 遍历 20 种氨基酸
    for aa in amino_acids:
        # 调用函数 (wt_aa 在函数内部获取)
        wt, _, _, score = calculate_mutation_score(target_protein, pos, aa, model, tokenizer, device)
        
        if aa == wt: 
            continue # 跳过同义突变
            
        if score > 0.0: # 只记录分数大于 0 的 (节省显示空间，可按需调整)
            mutation_code = f"{wt}{pos}{aa}"
            results.append({
                "Mutation": mutation_code,
                "Pos": pos,
                "WT": wt,
                "Mut": aa,
                "LLR_Score": score
            })

# 按照分数从高到低排序
results.sort(key=lambda x: x["LLR_Score"], reverse=True)

print(f"\\n🏆 Top 10 推荐突变 (Region {scan_start}-{scan_end}):")
print(f"{'Mutation':<10} | {'Score':<10}")
print("-" * 25)

for res in results[:10]:
    print(f"{res['Mutation']:<10} | {res['LLR_Score']:.4f}")

🔄 正在扫描区域 1 - 10 (1-based)...
\n🏆 Top 10 推荐突变 (Region 1-10):
Mutation   | Score     
-------------------------
V1W        | 0.6752
V1F        | 0.6234
V1L        | 0.6156
V1Q        | 0.6039
V1P        | 0.5775
V1Y        | 0.5765
V1A        | 0.5756
V1R        | 0.5599
V1S        | 0.5551
V1K        | 0.5546
